In [2]:
from langchain.chat_models import ChatOpenAI
from kor import create_extraction_chain, Object, Text 

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=2000,
    frequency_penalty=0,
    presence_penalty=0,
    top_p=1.0,
)

schema = Object(
    id="player",
    description=(
        "User is controlling a music player to select songs, pause or start them or play"
        " music by a particular artist."
    ),
    attributes=[
        Text(
            id="song",
            description="User wants to play this song",
            examples=[],
            many=True,
        ),
        Text(
            id="album",
            description="User wants to play this album",
            examples=[],
            many=True,
        ),
        Text(
            id="artist",
            description="Music by the given artist",
            examples=[("Songs by paul simon", "paul simon")],
            many=True,
        ),
        Text(
            id="action",
            description="Action to take one of: `play`, `stop`, `next`, `previous`.",
            examples=[
                ("Please stop the music", "stop"),
                ("play something", "play"),
                ("play a song", "play"),
                ("next song", "next"),
            ],
        ),
    ],
    many=False,
)

chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')
chain.run("play songs by paul simon and led zeppelin and the doors")['data']

In [ ]:
import json

def json_dump(json_object):
  json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False)
  print(json_formatted_str)

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
)


In [80]:
club_schema = Object(
    id="club",
    description="Information about a club",
    
    attributes=[
        Text(
            id="name",
            description="The name of the club."
        ),
        Text(
            id="manager",
            description="The name of the manager."
        ),
        Number(
            id="ranking",
            description="The ranking of the club."
        )
    ],
    examples=[
        (
            "Napoli managed by Spalletti are the 1st, Juventus managed by Allegri are the 5th.",
            [
                {"name": "Napoli", "manager": "Spalletti", "ranking": 1},
                {"name": "Juventus", "manager": "Allegri", "ranking": 5},
            ],
        )
    ]
)


In [81]:
extraction_chain = create_extraction_chain(llm, club_schema)

In [82]:

text='''
    Manchester City are just 1 point after Arsenal. 
    Arsenal are on top of the table.
    The 2 managers Pep and Arteta are going to meet face to face in this weekend.
'''
output = extraction_chain.predict_and_parse(text=text)['data']

json_dump(output)

{
  "club": [
    {
      "name": "Arsenal",
      "manager": "Arteta",
      "ranking": "1"
    },
    {
      "name": "Manchester City",
      "manager": "Pep",
      "ranking": "2"
    }
  ]
}


In [83]:
stock_market_schema = Object(
    id="stock_market",
    description="A股市场收评，上涨板块综述",
    
    # Notice I put multiple fields to pull out different attributes
    attributes=[
        Text(
            id="industry_name",
            description="行业板块名称"
        )
    ],
    examples=[
        (
            '''
                周二市场呈现出震荡调整格局，盘中行业板块及题材呈现局部轮动和分化，保险、煤炭、通信设备、银行、家用电器、IT设备、有色、电气设备等行业活跃。
                酒店餐饮、半导体、旅游、多元金融、电器仪表、元器件、通用机械、商业连锁、医药等行业呈现回调。
            ''',
            [
                {"industry_name": "保险"},
                {"industry_name": "煤炭"},
                {"industry_name": "通信设备"},
                {"industry_name": "银行"},
                {"industry_name": "家用电器"},
                {"industry_name": "IT设备"},
                {"industry_name": "有色"},
                {"industry_name": "电气设备"}
            ],
        ),
        (
            '''
                截至2023年4月14日收盘，上证指数收涨0.60%，报收3338.15点;深证成指涨0.51%，报收11800.09点;创业板指涨0.93%，报收2428.09点。

                沪深两市全天成交额11150亿元，北向资金净买入64.24亿元，其中沪股通净买入21.58亿元，深股通净买入42.66亿元。

                根据申万二级行业分类，2023年4月14日共有66个行业上涨，其中，能源金属、半导体、电子化学品、工业金属、小金属涨幅居前;

                58个行业下跌，酒店餐饮、旅游及景区、广告营销、互联网电商、数字媒体跌幅居前。

                2023年4月14日，沪深A股共有2655家公司上涨，46家公司涨幅超10%(含)，华星创业、耐科装备、联动科技、拓荆科技、经纬辉开涨幅居前;

                有2102家公司下跌，4家公司跌幅超10%(含)，登康口腔、值得买、奥飞娱乐、慈文传媒、金桥信息跌幅居前。

            ''',
            [
                {"industry_name": "能源金属"},
                {"industry_name": "半导体"},
                {"industry_name": "电子化学品"},
                {"industry_name": "工业金属"},
                {"industry_name": "小金属"}
            ],
        ),
        (
            '''
                传媒、游戏、数据确权、贵金属等板块涨幅居前，工业母机、白酒股等板块飘绿。
            ''',
            [
                {"industry_name": "传媒"},
                {"industry_name": "游戏"},
                {"industry_name": "数字确权"},
                {"industry_name": "贵金属"}
            ],
        )
    ]
)

In [84]:
text='''
    截至2023年4月13日收盘，上证指数收跌0.27%，报收3318.36点;深证成指跌1.21%，报收11739.84点;创业板指跌0.97%，报收2405.76点。

    沪深两市全天成交额11369亿元，北向资金净卖出8688万元，其中沪股通净买入20.64亿元，深股通净卖出21.51亿元。

    根据申万二级行业分类，2023年4月13日共有46个行业上涨，其中，旅游及景区、酒店餐饮、出版、教育、互联网电商涨幅居前;

    78个行业下跌，通信设备、半导体、元件、消费电子、黑色家电跌幅居前。
'''
extraction_chain = create_extraction_chain(llm, stock_market_schema)
output = extraction_chain.predict_and_parse(text=text)['data']

json_dump(output)

{
  "stock_market": [
    {
      "industry_name": "旅游及景区"
    },
    {
      "industry_name": "酒店餐饮"
    },
    {
      "industry_name": "出版"
    },
    {
      "industry_name": "教育"
    },
    {
      "industry_name": "互联网电商"
    }
  ]
}
